In [1]:
import geopandas as gpd
import pandas as pd
import os
#from shapely.geometry import LineString
from shapely import wkt
import numpy as np
from utils.load_csv import load_csv
from utils.load_shp import load_shp
from utils.save_shp import save_shp
#from TESTE 1:
def manual_data_path(file_name)->str:
    path = os.path.join(
        'data',
        'manual_data',
        file_name
    )
    return path

In [2]:
onibus_diesel_dia31= pd.read_csv(manual_data_path('linestring_2025-12-31_silver.csv'))

In [3]:
BUFFER = 10

In [4]:
df_onibus=onibus_diesel_dia31
df_onibus['geometry'] = df_onibus['geometry'].apply(wkt.loads) #transforma str em geometry
df_onibus.columns

Index(['codigo_onibus', 'geometry'], dtype='str')

Para transformar o df em gdf, precisamos setar a geometria... mas pra isso a coluna precisa ser do tipo geometria. Nossa coluna 'geometry' está como `str` atualmente, então o certo não é colocar como `set_geometry`, mas usar o `wkt` (well known text) do `shapely`

In [5]:
gdf_onibus =gpd.GeoDataFrame(
    df_onibus, 
    geometry= 'geometry',
    crs="EPSG:31983"
)

In [6]:
#escolher ônibus aleatórios

In [7]:
#transformar pontos x e y em geometry

In [8]:
#TRATAR DADOS
#gdf_onibus = gdf_onibus.sort_values(['codigo_onibus', 'hora_gps']).reset_index(drop = True)
#counts = gdf_onibus.codigo_onibus.value_counts()
#remover = counts[counts < 2].index
#gdf_onibus = gdf_onibus[~gdf_onibus.codigo_onibus.isin(remover)]

In [9]:
#LINHAS? linestring-> gdf_trajetos

In [10]:
gdf_buffer = gdf_onibus.sample(15).copy()
gdf_buffer

,codigo_onibus,geometry
5026,55466,"LINESTRING (345695.964 7389443.334, 346270.391..."
3598,45614,"LINESTRING (349769.448 7385603.452, 349769.448..."
8249,86076,"LINESTRING (320248.143 7396163.764, 320248.143..."
3969,47816,"LINESTRING (355589.134 7387863.224, 355492.808..."
5303,61745,"LINESTRING (326941.309 7373749.578, 326941.309..."
3822,47293,"LINESTRING (354283.204 7394460.792, 354283.204..."
2756,35721,"LINESTRING (344698.27 7398497.465, 345830.686 ..."
1872,26495,"LINESTRING (334676.083 7404864.119, 334209.188..."
7026,75500,"LINESTRING (331456.701 7392768.685, 331462.195..."
541,16087,"LINESTRING (328287.66 7404151.986, 328287.66 7..."


In [11]:
gdf_buffer['geometry'] = gdf_buffer.geometry.buffer(BUFFER)
gdf_buffer

,codigo_onibus,geometry
5026,55466,"POLYGON ((335579.771 7391573.462, 335579.764 7..."
3598,45614,"POLYGON ((346051.833 7392640.04, 346032.584 73..."
8249,86076,"POLYGON ((320231.026 7396134.108, 320231.026 7..."
3969,47816,"POLYGON ((350343.624 7395286.835, 350121.697 7..."
5303,61745,"POLYGON ((326651.807 7377200.667, 326537.171 7..."
3822,47293,"POLYGON ((354286.705 7394451.424, 354274.726 7..."
2756,35721,"POLYGON ((345826.184 7399077.393, 345826.982 7..."
1872,26495,"POLYGON ((334004.849 7400993.093, 334004.843 7..."
7026,75500,"POLYGON ((325525.325 7383017.084, 325525.325 7..."
541,16087,"POLYGON ((326981.145 7399891.787, 326981.133 7..."


In [12]:
#dissolve por codigo_onibus

# 04_overlay_setores_censitários

In [13]:
setores_censitarios_sp = load_shp('setores_censitarios_sp.shp')
setores_censitarios_sp = setores_censitarios_sp.to_crs(gdf_buffer.crs)

In [14]:
gdf_overlay = gpd.overlay(setores_censitarios_sp,
                          gdf_buffer,
                          how = 'intersection')
gdf_overlay = gdf_overlay.explode(index_parts = False).reset_index(drop = True)
len(gdf_overlay)

4994

In [15]:
#tirar geometrias pequenininhas
debuff_value= -(BUFFER**(1/2))
 
gdf_overlay['debuffed'] = gdf_overlay.buffer(debuff_value)

gdf_overlay['debuffed_area']= gdf_overlay['debuffed'].area

gdf_overlay = gdf_overlay[gdf_overlay['debuffed_area']>0]

gdf_overlay = gdf_overlay.drop(columns = {'debuffed', 'debuffed_area'})
len(gdf_overlay)

4524

# 05_calculo_pop_afetada

In [16]:
setores_censitarios_sp = setores_censitarios_sp.to_crs(gdf_overlay.crs)

setores_censitarios_sp['area_setor'] = setores_censitarios_sp.geometry.area

setores_area = setores_censitarios_sp[['CD_SETOR', 'area_setor', 'pop']]

In [17]:
gdf_overlay['area_intersecao'] = gdf_overlay.geometry.area

In [18]:
gdf_overlay = gdf_overlay.merge(setores_area,
                                on = 'CD_SETOR',
                                how = 'left')

In [19]:
gdf_overlay = gdf_overlay.drop(columns = ['pop_y'])

gdf_overlay = gdf_overlay.rename(columns = {'pop_x': 'pop'})

In [20]:
gdf_overlay['pop_afetada'] = round((gdf_overlay['pop'] * (gdf_overlay['area_intersecao'] / gdf_overlay['area_setor']))).astype(int)

In [21]:
gdf_overlay = gdf_overlay.rename(columns = {'area_intersecao': 'area_inter', 
                                            'pop_afetada': 'pop_buffer'})

In [22]:
gdf_overlay.head()

,CD_SETOR,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,...,v0003,v0004,v0005,v0006,v0007,codigo_onibus,geometry,area_inter,area_setor,pop_buffer
0,355030801000095P,0.063583,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,407,0,2.639485,9.871245,233,55466,"POLYGON ((340483.934 7391253.494, 340488.673 7...",1152.365903,63571.890784,11
1,355030801000186P,0.044869,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,204,0,2.161111,0.555556,180,55466,"POLYGON ((340913.678 7391322.72, 340841.167 73...",594.731182,44861.421604,5
2,355030801000187P,0.003456,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,120,0,2.389831,0.000000,118,55466,"POLYGON ((340959.339 7391322.759, 340958.987 7...",391.991719,3454.940422,32
3,355030801000188P,0.006967,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,179,0,2.446328,0.000000,177,55466,"POLYGON ((341029.7 7391311.253, 340958.987 739...",606.735282,6965.289963,38
4,355030801000200P,0.081879,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,229,0,2.097345,0.000000,226,55466,"POLYGON ((340786.04 7391321.115, 340706.621 73...",499.123442,81864.974142,3


In [23]:
#NÃO QUERO PENSAR NISSO AGORA
#setores_uniques = pd.DataFrame(gdf_overlay['CD_SETOR'].unique())

#setores_uniques.rename(columns={0:'name'}, inplace=True)
#for i, c in setores_uniques.iterrows():
 #   cd = c['name']
  #  setor_x = gdf_overlay.loc[gdf_overlay['CD_SETOR'] == cd]
   # c['soma_pop_afetada'] = sum(setor_x['pop_buffer'])
    #setores_uniques.loc[setores_uniques['name'] == cd, 'subtrac_pop'] = (setor_x.loc[setor_x['CD_SETOR'] == cd, 'pop']-c['soma_pop_afetada'])

#setores_uniques.sort_values(by = 'subtrac_pop', ascending = True).head(10)

In [24]:
#gdf_buffer['pop_afetada'] = [
#    gdf_overlay.loc[gdf_overlay['codigo_onibus'] == cb, 'pop_buffer'].sum()
#    for cb in gdf_buffer['codigo_onibus']
#]
gdf_buffer['pop_afetada'] = gdf_buffer['codigo_onibus'].map(
    gdf_overlay.groupby('codigo_onibus')['pop_buffer'].sum()
)
gdf_buffer

,codigo_onibus,geometry,pop_afetada
5026,55466,"POLYGON ((335579.771 7391573.462, 335579.764 7...",30579
3598,45614,"POLYGON ((346051.833 7392640.04, 346032.584 73...",16583
8249,86076,"POLYGON ((320231.026 7396134.108, 320231.026 7...",13372
3969,47816,"POLYGON ((350343.624 7395286.835, 350121.697 7...",12387
5303,61745,"POLYGON ((326651.807 7377200.667, 326537.171 7...",5709
3822,47293,"POLYGON ((354286.705 7394451.424, 354274.726 7...",20628
2756,35721,"POLYGON ((345826.184 7399077.393, 345826.982 7...",16156
1872,26495,"POLYGON ((334004.849 7400993.093, 334004.843 7...",11342
7026,75500,"POLYGON ((325525.325 7383017.084, 325525.325 7...",30529
541,16087,"POLYGON ((326981.145 7399891.787, 326981.133 7...",22029


# Média do dia

In [25]:
media_pop_dia = gdf_buffer['pop_afetada'].mean()
round(media_pop_dia, 0)

np.float64(17949.0)